<a href="https://colab.research.google.com/github/ximman/My-box/blob/main/%D0%98%D0%B7%D0%B2%D0%BB%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B8%D0%BC%D0%B5%D0%BD%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D1%81%D1%83%D1%89%D0%BD%D0%BE%D1%81%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install natasha #загружаем модель Наташа для работы с русскоязычными текстами

In [ ]:
import requests #для запросов на сайты (подгрузка приветствий, прощаний, имён)
import pandas as pd #работа с таблицами
import re #регулярные выражения для извлечения нужных отрезков текстов
from natasha import (     
    Segmenter,
    MorphVocab,  
    PER,
    NamesExtractor, #модули Наташи, которые нам пригодятся (морфологический анализ, извлечение имён, 
    NewsNERTagger,  #эмбеддинга и разметка по новостям, анализ синтаксиса)
    NewsEmbedding,
    NewsSyntaxParser,
    Doc
)
emb = NewsEmbedding()
segmenter = Segmenter()
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab) #создаём методы на основе подключенных модулей
syntax_parser = NewsSyntaxParser(emb)

from google.colab import drive
drive.mount('/content/drive') #подгружаем исходные данные с диска

Создаём список прощаний

In [ ]:
word_for_url = requests.utils.quote('до свидания') #проанализировав прощания, у «до свидания» самая адекватная выборка синонимов; забираем их в список 
url = 'http://www.synonymizer.ru/index.php?sword=' + word_for_url #формируем URL адрес                                                        прощаний      
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'} #беру из 
r = requests.get(url, headers = headers) #формируем запрос                                                                   инструмента разработчика 
r = r.text.splitlines() #делим ответ с сайта на строки для упрощения обработки                                                               браузера
list_1 = [] #создаём список, в который будем записывать нужные строки
for i in range(len(r)):
  if 'База синонимов' in r[i]: #по структуре вернувшейся страницы извлекаем синонимы с трёх баз синонимов
    while r[i] != '</div>': #этими символами заканчивается блок с синонимами с трёх баз
      list_1.append(r[i+1])
      i += 1
s = [] #создаём список, в который будем складывать индексы пустых элементов списка синонимов, чтобы потом удалить их
for l in range(len(list_1)):
  list_1[l] = list_1[l].replace('<p>', '') #
  list_1[l] = list_1[l].replace('</p>', '') #чистим список прощаний от ненужных нам символов
  list_1[l] = list_1[l].replace('</div>', '') #
  if list_1[l] == '': #определяем пустой элемент
    s.append(l) #записываем в расстрельный список его порядковый номер
for j in reversed(s): #считываем с обратной стороны, чтобы удалённые элементы не сдвигали нумерацию непроанализированных элементов
  list_1.pop(j) #выкидываем пустой элемент, который может появиться, если одна из баз синонимов пустая
string = '' #создаём строку, в которую будем объединять список синонимов, элементы которого на данный момент склеенные
for k in range(len(list_1)):
  if k > 0:
    string += ', '+ list_1[k] #преобразуем список в строку
  else: string += list_1[k]
list_2 = string.split(', ') #разделяем строку по запятой в список синонимов
for i in reversed(range(len(list_2))): #проверяем строку на наличие скобок с конца, чтобы не нарушать нумерацию непроверенных элементов
  if '(' in list_2[i] or ')' in list_2[i]: #разобраемся со скобками, например, в списке есть элемент «здравствуй(те)», который никогда не 
    list_2.pop(i) #                                                                                                 встретится в диалоге
list_bye = list(set(list_2)) #убираем повторяющиеся синонимы в списке

Ручной ввод прощания в список прощаний

In [ ]:
flag_repeat = False #метка, показывающая, что добавляемое прощание уже есть в списке
add_bye = input('Введите прощание, которое нужно добавить в список: ').lower() #вводим прощание и преобразуем в строчные буквы, потому что весь 
for i in range(len(list_bye)): #проводим перебор по всем прощаниям                                                  список прощаний из строчных
  if add_bye == list_bye[i]: #при условии совпадения добавляемого прощания с элементом списка
    print('Прощание "', add_bye, '" уже есть в списке прощаний', sep = '')
    flag_repeat = True #если совпадение, то устанавливаем флаг
    break #прерываем цикл сравнения
if flag_repeat == False: #если во всём списке нет совпадений, то выполняем условие
  list_bye.append(add_bye)
  print('Прощание "', add_bye, '" добавлено в список прощаний', sep = '')

Ручное удаление прощания из списка прощаний

In [ ]:
flag_repeat = False #аналогично предыдущему блоку метка на повтор
del_bye = input('Введите прощание, которое нужно удалить из списка: ').lower() #вводим прощание, которое, по нашему мнению, лишнее в списке
for i in range(len(list_bye)): #перебираем весь список прощаний
  if del_bye == list_bye[i]: #при совпадении вводимого прощания с элементом списка
    flag_repeat = True #при совпадении меням метку
    list_bye.pop(i) #удаляем совпавший элемент списка
    print('Прощание "', del_bye, '" удалено из списка прощаний', sep = '')
    break #выходим из цикла перебора
if flag_repeat == False: #если метка не изменилась, значит совпадений не было
  print('Прощание "', del_bye, '" не найдено в списке прощаний', sep = '')

Создаём список приветствий

In [ ]:
word_for_url = requests.utils.quote('доброе утро') #по аналогии с прощанием кодируем самое подходящее по синонимам приветствие для URL запроса
url = 'http://www.synonymizer.ru/index.php?sword=' + word_for_url #формируем URL адрес
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'} #беру 
r = requests.get(url, headers = headers) #записываем в переменную ответ на запрос                           из инструмента разработчика браузера
r = r.text.splitlines() #разбиваем полученную массу текста на строки
list_1 = []
for i in range(len(r)):
  if 'База синонимов' in r[i]: #снова по ключевым словам определяем начало базы синонимов
    while r[i] != '</div>':
      list_1.append(r[i+1])
      i += 1
s = []
for l in range(len(list_1)):
  list_1[l] = list_1[l].replace('<p>', '')
  list_1[l] = list_1[l].replace('</p>', '')
  list_1[l] = list_1[l].replace('</div>', '') #чистим полученные синонимы от посторонних символов
  if list_1[l] == '': 
    s.append(l)
for j in reversed(s):
  list_1.pop(j)
string = ''
for k in range(len(list_1)):
  if k > 0:
    string += ', '+ list_1[k]
  else: string += list_1[k]
list_2 = string.split(', ')
for i in reversed(range(len(list_2))):
  if '(' in list_2[i] or ')' in list_2[i]: #все шаги по подобию создания списка прощаний
    list_2.pop(i)
list_hello = list(set(list_2))

Ручное добавление приветствия в список приветствий

In [ ]:
flag_repeat = False
add_hello = input('Введите приветствие, которое нужно добавить в список: ').lower()
for i in range(len(list_hello)):
  if add_hello == list_hello[i]:
    print('Приветствие "', add_hello, '" уже есть в списке приветствий', sep = '') #все шаги схожи со списком прощаний
    flag_repeat = True
    break
if flag_repeat == False:
  list_hello.append(add_hello)
  print('Приветствие "', add_hello, '" добавлено в список приветствий', sep = '')

Ручное удаление приветствия из списка приветствий

In [ ]:
flag_repeat = False
del_hello = input('Введите приветствие, которое нужно удалить из списка: ').lower()
for i in range(len(list_hello)):
  if del_hello == list_hello[i]:
    flag_repeat = True
    list_hello.pop(i)
    print('Приветствие "', del_hello, '" удалено из списка приветствий', sep = '') #все шаги схожи со списком прощаний
    break
if flag_repeat == False:
  print('Приветствие "', del_hello, '" не найдено в списке приветствий', sep = '')

Создаём словарь имён

In [ ]:
list_names = []
headers_names = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
for letter in ('абвгдежзиклмнопрстуфхцчшэюя'): #перебираю буквы, чтобы перейти на страницу с именами на них
  letter_for_url = requests.utils.quote(letter) #преобразую букву для URL запроса
  url_names = 'https://lexicography.online/onomastics/superanskaya/' + letter_for_url #добавляю зашифрованную букву к адресу
  r = requests.get(url_names, headers = headers_names) #записываю в перемненную ответ на запрос
  list_names += (re.findall('[(А-ЯЁа-яё)]+', r.text.splitlines()[25])) #в 25 строке сожержатся имена, но вперемешку со служебными символами, 
  #                                                                                       которые отбрасываю с помощью регулярных выражений

Ручное добавление имени в список имён

In [ ]:
flag_repeat = False
add_name = input('Введите имя, которое нужно добавить в список: ').title() #все шаги аналогичны изменению списка прощаний
for i in range(len(list_names)):
  if add_name == list_names[i]:
    print('Имя "', add_name, '" уже есть в списке имён', sep = '')
    flag_repeat = True
    break
if flag_repeat == False:
  list_names.append(add_name)
  print('Имя "', add_name, '" добавлено в список имён', sep = '')

Ручное удаление имени из списка имён

In [ ]:
flag_repeat = False
del_name = input('Введите имя, которое нужно удалить из имён: ').lower() #все шаги аналогичны изменению списка прощаний
for i in range(len(list_names)):
  if del_name == list_names[i]:
    flag_repeat = True
    list_names.pop(i)
    print('Имя "', del_name, '" удалено из списка имён', sep = '')
    break
if flag_repeat == False:
  print('Имя "', del_name, '" не найдено в списке имён', sep = '')

Загружаем таблицу с диска в переменную

In [ ]:
data_original = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test_data.csv') #создаём датафрейм из csv файла на диске
data = data_original.copy() #создаю копию, чтобы не изменить оригинальную таблицу

Ищем фразы, в которых было приветствие

In [ ]:
data['status'] = '' #создаём столбец для наших пометок
num_lines = 5 #количество строк с начала и конца диалога, в которых будем искать нужные нам сущности (дальше 5 строк реплик не встречались)
for dlg_id in range(data['dlg_id'].max() + 1): #перебираем номера диалогов (максимальное значение +1 так как range начинает с 0)
  for line_n in range(num_lines):
    for hello in list_hello:
      buffer = data[(data['dlg_id'] == dlg_id) & (data['role'] == 'client')]['text'].iloc[line_n].lower() #ищем в пределах одного диалога и роли   
      if hello in buffer: #при совпадении приветствия с элементом реплики                                 #клиента (ошибка расшифровывающей системы)
        data.loc[(data['text'] == buffer.capitalize()) & (data['dlg_id'] == dlg_id),'status'] = 'hello' #записываем в наш столбец метку для реплики 
        break #если нашли в реплике приветствие, то переходим к следующей реплике                                                    с приветствием

Ищем фразы, в которых было прощание

In [ ]:
for dlg_id in range(data['dlg_id'].max() + 1):
  for line_n in range(num_lines):
    for bye in list_bye:
      buffer = data[(data['dlg_id'] == dlg_id) & (data['role'] == 'client')]['text'].iloc[-line_n].lower() #всё по аналогии с предыдущим блоком
      if bye in buffer:
        data.loc[(data['text'] == buffer.capitalize()) & (data['dlg_id'] == dlg_id),'status'] = 'bye'
        break

Ищем имена во фразах и устанавливаем заглавную букву для повышения узнаваемости Natasha (предустановленные алгоритмы не работали со строчными буквами). После этого ищем выделенные заглавной буквой имена с помощью Natasha

In [ ]:
'''Строчные буквы вызвали некоторые проблемы с распознаванием, почему и проишлось делать большую надстройку в виде списка имён, например. Методы title(), 
capitalize() и upper() иногда улучшали распознаваемость, но результат был неудовлетворительный. Методом изысканий нашлось решение, но с перебором имён 
занимающее приличное время. Можно было бы запрашивать имя менеджера вручную, но такое решение пришлось не по вкусу. Имеем то, что имеем на данный момент. 
В будущем решу лучше'''
for dlg_id in range(data['dlg_id'].max() + 1): #проходимся по номерам диалгов
  for line_n in range(num_lines): #берём нужное нам количество строк
    for name in list_names: #перебираем  имена из списка имён
      name = name.lower() #приводим имя к строчным буквам, так как в списке с заглавной, а в репликах всё строчными
      buffer = data[(data['dlg_id'] == dlg_id) & (data['role'] == 'client')]['text'].iloc[line_n].lower().split(' ') #берём нужную нам реплику и 
      for count in range(len(buffer)): #перебираем элементы созданного списка                                               разделяем по пробелу
        if buffer[count] == name: #при нахождении имени в списке
          if 'зовут' in ' '.join(buffer) or 'это' in ' '.join(buffer): #проверяем на признаки того, что названное имя — имя менеджера, а не, например, 
            buffer[count] = name.capitalize() #пишем имя в реплику с заглавной буквы                                                        обращение
            doc = Doc(' '.join(buffer)) #создаём файл для обработки Natasha
            doc.segment(segmenter) #разделяем строку
            doc.tag_ner(ner_tagger) #пытаемся разметить на сущности
            for span in doc.spans:
                span.normalize(morph_vocab) #нормализуем элементы
            for span in doc.spans:
              if span.type == 'PER': #берём элементы, которые были распознаны как имя
                  span.extract_fact(names_extractor) #извлекаем имя
                  data.loc[(data['text'] == ' '.join(buffer).lower().capitalize()) & 
                           (data['dlg_id'] == dlg_id),'status'] = 'name: ' + name.title() + ' '
                  #записываем для строки, где нашлось представление менеджером себя, в столбец status метки и имени с заглавной буквы, добавив пробел 
                  #                                                                                                  для возможного названия компании

Подготавливаем фразу и анализируем её на предмет названия организации с помощью всё той же Natasha

In [ ]:
list_rel = ['amod', 'nmod', 'nsubj', 'nsubjpass', 'appos'] #серия экспериментов показала, что из этих частей речи чаще всего состоят названия 
for dlg_id in range(data['dlg_id'].max() + 1): #уже знакомый нам перебор по номерам диалогов                                       организаций
  for line_n in range(num_lines): #и количеству строк с начала разговора
    text = data[(data['dlg_id'] == dlg_id) & (data['role'] == 'client')]['text'].iloc[line_n] #записываем реплики менеджера в переменную
    list_org = []
    doc1 = Doc(text) #реплику преобразуем в удобный для Natasha вид
    doc1.segment(segmenter) #разделяем реплику
    doc1.parse_syntax(syntax_parser) #проводим синтакический анализ (нас интересуют части речи, но взаимосвввязи слов тоже интересно рассмотреть)
    for i in range(len(doc1.tokens)):
      if 'компани' in doc1.tokens[i].text.lower(): #начиная со слова «компания» в разных склонениях начинаем записывать название организации
        for j in range(i + 1, len(doc1.tokens)):
          if 'называет' in doc1.tokens[j].text or 'именует' in doc1.tokens[j].text: #после слова «компания» может быть вставлено «называется», учтём 
            continue #переходим на следующий виток цикла, чтобы не записать лишнее слово в название                                   и такой вариант
          if doc1.tokens[j].rel in list_rel: #пока слова входят в список допустимых частей речи, мы записываем их в название
            list_org.append(doc1.tokens[j].text)
          else: 
            data.loc[data['text'] == text,'status'] += ('org: ' + ' '.join(list_org).title()) 
            break #если слово не той части речи, то записываем в столбец ststus метку и название компании, завершаем цикл проверки на части речи
        break 

Выводим удобно читаемые данные по результатам анализа расшифровок телефонных разговоров

In [ ]:
for dlg_id in range(data['dlg_id'].max() + 1):
  data_1 = data[(data['dlg_id'] == dlg_id) & (data['status'] != '')] #отсеиваем пустые ячейки столбца status для нужного диалога, чтобы не тратить 
  for i in range(len(data_1.index)): #                                                                                                на них время
    if 'name' in data_1.iloc[i, 4]: #проверяем на наличие метки имени в столбце status
      print('\nМенеджер диалога № ', dlg_id, ' ', data_1.iloc[i, 4].split(' ')[1], '.', sep = '') #делим всю запись по пробелу и берём 1 элемент, 
      print('\nПредставился(-лась) в репллике: ', data_1.iloc[i, 3], sep = '') #                                       на котором всегда будет имя
      break
    elif i == len(data_1.index) - 1: 
      print('\nМенеджер диалога № ', dlg_id, ' не представился(-лась).', sep = '')
  
  flag_hello = False #метка на наличие приветствия
  flag_bye = False #метка на наличие прощания
  for j in range(len(data_1.index)): 
    if 'hello' in data_1.iloc[j, 4]: #проверяем на наличие метки приветствия в столбце status
      if flag_hello == False:
        print('\nПоздоровался(-лась) в реплике:')
      print(data_1.iloc[j, 3]) #выводим реплику нужной строки таблицы
      flag_hello = True #устанавливаем метку наличия приветствия
    if 'bye' in data_1.iloc[j, 4]: #проверяем на наличие метки прощания в столбце status
      if flag_bye == False:
        print('\nПопрощался(-лась) в реплике:')
      print(data_1.iloc[j, 3]) #выводим реплику нужной строки таблицы
      flag_bye = True #устанавливаем метку наличия прощания
    if flag_hello == True and flag_bye == True and j == len(data_1.index) - 1: #условие с j нужно, чтобы убедиться в том, что проверены все строки
      print('\nТребование поздороваться и попрощаться ВЫПОЛНЕНО', sep = '')
    elif flag_hello == True and flag_bye == False and j == len(data_1.index) - 1:
      print('\nТребование поздороваться и попрощаться НЕ ВЫПОЛНЕНО: менеджер не попрощался(-лась)', sep = '')
    elif flag_hello == False and flag_bye == True and j == len(data_1.index) - 1:
      print('\nТребование поздороваться и попрощаться НЕ ВЫПОЛНЕНО: менеджер не поздоровался(-лась)', sep = '')
    elif flag_hello == False and flag_bye == False and j == len(data_1.index) - 1:
      print('\nТребование поздороваться и попрощаться НЕ ВЫПОЛНЕНО: менеджер не поздоровался и не попрощался(-лась)', sep = '')
  
  for n in range(len(data_1.index)):
    if 'org' in data_1.iloc[i, 4]: #проверяем на наличие метки названия компании в столбце status
      print('\nПредставил(-а) компанию как "', ' '.join(data_1.iloc[i, 4].split(' ')[3:]), '".', sep = '') #делим всю запись по пробелу и берём  
      break #прерываем цикл после того, как нашли представление компании                          элементы,на которых всегда будет имя (до конца)
    elif n == len(data_1.index) - 1: 
      print('\nНе представил(-а) компанию', sep = '')
  print('__________________________________')


Менеджер диалога № 0 Ангелина.

Представился(-лась) в репллике: Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается

Поздоровался(-лась) в реплике:
Алло здравствуйте

Попрощался(-лась) в реплике:
Всего хорошего до свидания

Требование поздороваться и попрощаться ВЫПОЛНЕНО

Представил(-а) компанию как "Диджитал Бизнес".
__________________________________

Менеджер диалога № 1 Ангелина.

Представился(-лась) в репллике: Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается

Поздоровался(-лась) в реплике:
Алло здравствуйте

Попрощался(-лась) в реплике:
Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего
До свидания

Требование поздороваться и попрощаться ВЫПОЛНЕНО

Представил(-а) компанию как "Диджитал Бизнес".
__________________________________

Менеджер диалога № 2 Ангелина.

Представился(-лась) в ре